In [8]:
import pandas as pd
import numpy as np
import scipy.sparse 
from scipy.sparse import csr_matrix
import math
import sparse
import ast
from tqdm import tqdm
import time

import gurobipy as gp
from gurobipy import GRB


def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [9]:
#read excel files and put in correct format
#Voorbeeldje met 5 zones
# dataframe = pd.read_csv(r'C:\Users\soudeveldhuis\Documents\Voorbeeldje.csv', sep=';', keep_default_na=False)
# obsMatrix = pd.read_csv(r'C:\Users\soudeveldhuis\Documents\voorbeeldjeObs.txt', sep=';', header=None)

#Voorbeeld met 20 zones
# dataframe = pd.read_csv(r'C:\Users\soudeveldhuis\Documents\Modellen\Zwolle2\Zwolle_2019_Autovracht_v16\populationAfterOctavius.csv',keep_default_na=False)
# obsMatrix = pd.read_csv(r'C:\Users\soudeveldhuis\Documents\Modellen\Zwolle2\ObservedMatrix.txt', sep=';', header=None)
start_time=time.time()
#Grote instantie
dataframe = pd.read_csv(r'C:\Users\soudeveldhuis\Documents\Modellen\Zwolle4\Zwolle_2019_Autovracht_v16\populationAfterOctavius.csv',keep_default_na=False, float_precision='round_trip')
obsMatrix = pd.read_csv(r'C:\Users\soudeveldhuis\Documents\Modellen\Zwolle4\NormObservedMatrix.txt', sep=';', header=None, float_precision='high')
# print(obsMatrix.dtypes)
# print(obsMatrix.sum())
# print(sum(obsMatrix))

# for i in tqdm(range(0,len(obsMatrix))):
#     for j in range(0,len(obsMatrix)):
#         obsMatrix[i][j]=float(obsMatrix[i][j])
# print(obsMatrix.dtypes)
# print(obsMatrix.sum)

In [10]:
# print(obsMatrix[6][66])
# colsum=obsMatrix.sum()
# rowsum=obsMatrix.sum(axis=1)
# print(colsum)
# print(rowsum)
# print(np.sum(np.sum(obsMatrix)))
# print(colsum.sum())
# print(rowsum.sum())

In [11]:
#Change the database, so we have only the columns that we need
#Change destinations from strings to integers and remove rows that have no destinations or destinations that are not inside our zones
#Delete probabilities lowe than 5%

usedlist=['agentID', 'Woonzone', 'Vervoerswijze', 'Nevenbestemming_auto', 'Volgorde', 'Hoofdbestemming_auto', 'AantalTrips']
zonenr = max(dataframe['Woonzone'])
maxagentnr = max(dataframe['agentID'])
# print(maxagentnr)
tester=dataframe.copy()
# print(tester)

tester['Hoofdbestemming_auto']=tester['Hoofdbestemming_auto'].replace('',0).astype('int')
tester['Nevenbestemming_auto']=tester['Nevenbestemming_auto'].replace('',0).astype('int')
tester['TweedeHoofdbestemming_auto']=tester['TweedeHoofdbestemming_auto'].replace('',0).astype('int')

hoofdrows=tester[tester['Hoofdbestemming_auto'] == 0].index
nevenrows=tester[tester['Nevenbestemming_auto'] == 0].index

verweg=tester[tester['Hoofdbestemming_auto'] > zonenr].index
verwegneven=tester[tester['Nevenbestemming_auto'] >zonenr].index

tourtweerows=tester[tester['TweedeHoofdbestemming_auto']>0].index
tourtweeverweg=tester[tester['TweedeHoofdbestemming_auto'] <= zonenr].index
twotouragents=intersection(tourtweerows,tourtweeverweg)
# print(tester['agentID'])
# newrow={'agentID': [range(twotouragents)], 'Woonzone': [], 'Hoofdbestemming_auto': [], 'Vervoerswijze':[], 'AantalTrips': []}
newrow={}
k=0
for t in tqdm(twotouragents, desc="Splitting two tours"):
    k+=1
    agentId=maxagentnr+k
    woonzone=tester['Woonzone'][t]
    hoofdbestemming=tester['TweedeHoofdbestemming_auto'][t]
    vervoerswijze=tester['VervoerswijzeTour2'][t]
    # print(f'agentid {agentId}, woonzone {woonzone}, hoofdbestemming {hoofdbestemming}, vervoerswijze {vervoerswijze}')
    # newrow={'agentID': [agentId], , 'Hoofdbestemming_auto':[hoofdbestemming], 'Vervoerswijze':[vervoerswijze], 'AantalTrips': [2]}
    if k==1:
        newrow['agentID']=[agentId]
        newrow['Woonzone']=[woonzone]
        newrow['Hoofdbestemming_auto']=[hoofdbestemming]
        newrow['Vervoerswijze']=[vervoerswijze]
        newrow['AantalTrips']=[2]
    else:
        newrow['agentID'].append(agentId)
        newrow['Woonzone'].append(woonzone)
        # newrow['Woonzone'][agentId]=woonzone
        newrow['Hoofdbestemming_auto'].append(hoofdbestemming)
        newrow['Vervoerswijze'].append(vervoerswijze)
        newrow['AantalTrips'].append(2)
# print(newrow)
tester = pd.concat((tester,pd.DataFrame(newrow)), ignore_index=True)

tester['Nevenbestemming_auto']=tester['Nevenbestemming_auto'].fillna(0)
tester['Hoofdbestemming_auto']=tester['Hoofdbestemming_auto'].astype('int')
tester['Nevenbestemming_auto']=tester['Nevenbestemming_auto'].astype('int')


emptyrows=intersection(hoofdrows,nevenrows)
verwegrows=intersection(verweg,verwegneven)

test=tester[usedlist]
test = test.drop(emptyrows)
test = test.drop(verwegrows)


agents=list(test['agentID'])
# print(agents)
# print(test['Vervoerswijze'])
agentnr=len(agents)
zonenr = max(test['Woonzone'])
zones = list(range(zonenr))

for a in tqdm(agents, desc="Turning vervoerswijze into string"):
    a=a-1
    if len(test['Vervoerswijze'][a]) > 2:
        newstring=ast.literal_eval(str(test['Vervoerswijze'][a].replace('=>',':')))
        test['Vervoerswijze'][a]=newstring
    # if len(str.strip(test['VervoerswijzeTour2'][a])):
    #     ttstring=ast.literal_eval(str(test['VervoerswijzeTour2'][a].replace('=>',':')))
    #     test['VervoerswijzeTour2'][a]=ttstring
# print(test)

# smallprob1=[]
# smallprob2=[]

# for a in tqdm(agents, desc="vervoerswijze"):
#     if 'auto' in test['Vervoerswijze'][a-1]:
#         if test['Vervoerswijze'][a-1]['auto'] < 0.05:
#             smallprob1.append(a-1)
#     if 'auto' in test['VervoerswijzeTour2'][a-1]:
#         if test['VervoerswijzeTour2'][a-1]['auto'] < 0.05:
#             smallprob2.append(a-1)
# smallprob=smallprob1.copy()
# smallprobb=smallprob2.copy()


# for a in smallprob1:
#     if 'auto' in test['VervoerswijzeTour2'][a]:
#         test['Vervoerswijze'][a]['auto']=0
#         smallprob.remove(a)
# for a in smallprob2:
#     if 'auto' in test['Vervoerswijze'][a]:
#         test['VervoerswijzeTour2'][a]['auto']=0
#         smallprobb.remove(a)

# test.drop(smallprob)
# test.drop(smallprobb)

test.to_csv(r'C:\Users\soudeveldhuis\Documents\Modellen\Zwolle4\AdaptedData.csv')


Turning vervoerswijze into string:   0%|          | 0/200507 [00:00<?, ?it/s]C:\Users\soudeveldhuis\AppData\Local\Temp\ipykernel_9244\3789425402.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Vervoerswijze'][a]=newstring
Turning vervoerswijze into string: 100%|██████████| 200507/200507 [06:42<00:00, 497.68it/s]


In [12]:
# Set indices and paramaters


df=test
ag=0
prob=agents.copy()
sparsematrix=csr_matrix((zonenr,zonenr))
usedagents={}
for i in zones:
    for j in zones:
        usedagents[(i,j)]=[]


for a in tqdm(agents, desc="Creating the probability matrix"):
    prob[ag]=sparsematrix.copy()
    a=a-1
    if 'auto' in df['Vervoerswijze'][a]:
        if df['Nevenbestemming_auto'][a] !=0:
            if 'hoofdmotief' in df['Volgorde'][a]:
                probability=df['Vervoerswijze'][a]['auto']
                origin=df['Woonzone'][a]
                firstdestination=df['Hoofdbestemming_auto'][a]
                seconddestination=df['Nevenbestemming_auto'][a]
                if firstdestination<=zonenr:
                    prob[agents.index(a+1)][origin-1,firstdestination-1]+=probability
                    if a+1 not in usedagents[(origin-1,firstdestination-1)]:
                        usedagents[(origin-1,firstdestination-1)].append(a+1)
                    if seconddestination<=zonenr-1:
                        prob[agents.index(a+1)][firstdestination-1, seconddestination-1]+=probability
                        if a+1 not in usedagents[(firstdestination-1, seconddestination-1)]:
                            usedagents[(firstdestination-1, seconddestination-1)].append(a+1)
                        prob[agents.index(a+1)][seconddestination-1,origin-1]+=probability
                        if a+1 not in usedagents[(seconddestination-1,origin-1)]:
                            usedagents[(seconddestination-1,origin-1)].append(a+1)
                elif seconddestination<=zonenr-1:
                    prob[agents.index(a+1)][seconddestination-1,origin-1]+=probability
                    if a+1 not in usedagents[(seconddestination-1,origin-1)]:
                        usedagents[(seconddestination-1,origin-1)].append(a+1)
            elif 'nevenmotief' in df['Volgorde'][a]:
                probability=df['Vervoerswijze'][a]['auto']
                origin=df['Woonzone'][a]
                firstdestination=df['Nevenbestemming_auto'][a]
                seconddestination=df['Hoofdbestemming_auto'][a]
                if firstdestination<=zonenr:
                    prob[agents.index(a+1)][origin-1,firstdestination-1]+=probability
                    if a+1 not in usedagents[(origin-1,firstdestination-1)]:
                        usedagents[(origin-1,firstdestination-1)].append(a+1)
                    if seconddestination<=zonenr:
                        prob[agents.index(a+1)][firstdestination-1,seconddestination-1]+=probability
                        prob[agents.index(a+1)][seconddestination-1,origin-1]+=probability
                        if a+1 not in usedagents[(firstdestination-1,seconddestination-1)]:
                            usedagents[(firstdestination-1,seconddestination-1)].append(a+1)
                        if a+1 not in usedagents[(seconddestination-1,origin-1)]:
                            usedagents[(seconddestination-1,origin-1)].append(a+1)
                elif seconddestination<=zonenr:
                    prob[agents.index(a+1)][seconddestination-1,origin-1]+=probability
                    if a+1 not in usedagents[(seconddestination-1,origin-1)]:
                        usedagents[(seconddestination-1,origin-1)].append(a+1)        
        elif df['Hoofdbestemming_auto'][a] != 0:
            probability=df['Vervoerswijze'][a]['auto']
            origin=df['Woonzone'][a]
            destination=df['Hoofdbestemming_auto'][a]
            if destination <= zonenr:
                # print(f'origin {origin}, destination {destination}, check type: {origin.dtype}, {destination.dtype}')
                prob[agents.index(a+1)][origin-1,destination-1]+=probability
                prob[agents.index(a+1)][destination-1,origin-1]+=probability
                if a+1 not in usedagents[(origin-1,destination-1)]:
                    usedagents[(origin-1,destination-1)].append(a+1)
                if a+1 not in usedagents[(destination-1,origin-1)]:
                    usedagents[(destination-1,origin-1)].append(a+1)
    # if df['AantalTours'][a]==2:
    #     if (df['VervoerswijzeTour2'][a]) != 0:
    #         if 'auto' in df['VervoerswijzeTour2'][a]:
    #             probability=df['VervoerswijzeTour2'][a]['auto']
    #             origin=df['Woonzone'][a]
    #             destination=df['TweedeHoofdbestemming_auto'][a]
    #             if destination <= zonenr:
    #                 prob[agents.index(a+1)][origin-1,destination-1]+=probability
    #                 prob[agents.index(a+1)][destination-1,origin-1]+=probability
    #                 if a+1 not in usedagents[(origin-1,destination-1)]:
    #                     usedagents[(origin-1,destination-1)].append(a+1)
    #                 if a+1 not in usedagents[(destination-1,origin-1)]:
    #                     usedagents[(destination-1,origin-1)].append(a+1)
    ag+=1

beforemodel_time=time.time()
# print(usedagents[(2,3)])
# print(obsMatrix)
# for i in agents:
#     print(f'probability for agent {i} is {prob[agents.index(i)]}')

# print(obsMatrix[2][3])
# print(prob[0][0,3])

          

Creating the probability matrix:   0%|          | 0/200507 [00:00<?, ?it/s]c:\Users\soudeveldhuis\Anaconda3\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
Creating the probability matrix: 100%|██████████| 200507/200507 [08:25<00:00, 396.80it/s]


In [13]:
# absolutematrix=np.zeros((696,696))
# # print(zones)
# for i in tqdm(zones):
#     for j in zones:
#         # print(f'i is {i}, j is {j}')
#         difference=sum((prob[agents.index(a)][i,j]) for a in usedagents[(i,j)])-obsMatrix[i][j]
#         if difference<0:
#             absolutematrix[i][j]=-difference
#         else:
#             absolutematrix[i][j]=difference
# print(sum(sum(absolutematrix)))

In [14]:
%store agents
%store usedagents
%store zones
%store prob
%store obsMatrix
%store maxagentnr
%store agents
%store beforemodel_time
%store start_time


Stored 'agents' (list)
Stored 'usedagents' (dict)
Stored 'zones' (list)
Stored 'prob' (list)
Stored 'obsMatrix' (DataFrame)
Stored 'maxagentnr' (int)
Stored 'agents' (list)
Stored 'beforemodel_time' (float)
Stored 'start_time' (float)
